## Deployment of Object Detection Model to Azure ML using MLFlow API

In [ ]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()

# mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


In [2]:
experiment_name = 'mlflow-aml-deployment'
mlflow.set_experiment(experiment_name)

In [ ]:
#Save the python flavor of the model together with environment spec
import mlflow.pyfunc
from sys import version_info

from pyimagesearch import config
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import pip
import pickle
import cv2
import os
import tensorflow as tf
import sklearn

# Construct and save the model
PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)
# artifacts = {
#     "model_path": "output"
# }

conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
            'imutils',
            'opencv-python-headless',
            'pillow',
            'azureml-mlflow',
          'tensorflow=={}'.format(tf.__version__),
          'scikit-learn=={}'.format(sklearn.__version__),
        ],
      },
    ],
    'name': 'tf_env'
}


model_path = "model"
mlflow.pyfunc.save_model(path=model_path, data_path ='output',loader_module ='pyimagesearch.model_loader', code_path=['pyimagesearch'], conda_env=conda_env)



### Preparing data to test service


In [7]:
#Testing the deployed service

from io import BytesIO
import json
import base64
ENCODING = 'utf-8'
test_image_paths =['dataset/images/airplane/image_0001.jpg','dataset/images/airplane/image_0002.jpg']
base64_string_list=[]
for test_image_path in test_image_paths:
    with open(test_image_path,"rb") as img:
        image_bytes = BytesIO(img.read())
    encoded_image =base64.b64encode(image_bytes.getvalue())
    base64_string = encoded_image.decode(ENCODING)
    base64_string= "b'{0}'".format(base64_string)
    base64_string_list.append(base64_string)
image_request = {"data": base64_string_list}
# payload.append(image_request)

body = json.dumps(image_request)

In [8]:
from mlflow.pyfunc.scoring_server import parse_json_input, _get_jsonable_obj

import json
# data = json.loads(body)['data']
out = parse_json_input(json_input=body, orient="split")


In [7]:
#Reload model locally and score 

import mlflow
model_path = "model"
loaded_model = mlflow.pyfunc.load_model(model_path)

pred_output = loaded_model.predict(out)
print(pred_output)

### Deploy model to ACI

In [9]:
from azureml.core.webservice import AciWebservice, Webservice

# Set the model path to the model folder created by your run

# Configure 
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                tags={'method' : 'keras/tf'}, 
                                                description='aml-mlflow model',
                                                location='westus2')


In [10]:
import mlflow.azureml
(webservice,model) = mlflow.azureml.deploy( model_uri=model_path,
                      workspace=ws,
                      model_name='object_detection_model', 
                      service_name='aml-mlflow-test-v2', 
                      deployment_config=aci_config, 
                      tags=None, mlflow_home=None, synchronous=True)

webservice.wait_for_deployment(show_output=True)


Registering model object_detection_model


2021/01/19 10:49:34 INFO mlflow.azureml: Registered an Azure Model with name: `object_detection_model` and version: `19`
2021/01/19 10:49:37 INFO mlflow.azureml: Deploying an Azure Webservice with name: `aml-mlflow-test-v2`


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


### Deploy model to AKS

In [ ]:
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
   prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_PB6s",
                                                       agent_count = 1,
                                                       location = "eastus")



In [29]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (can also provide parameters to customize)
# prov_config = AksCompute.provisioning_configuration()

prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_D11_v2",
                                                   agent_count = 3,
                                                   location = "westus2")

aks_name = 'aks-mlflow-d11'

# Create the cluster
aks_target = ComputeTarget.create(workspace=ws, 
                                  name=aks_name, 
                                  provisioning_configuration=prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)


Creating...............................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


In [40]:
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (using default here with app insights)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True, cpu_cores = 1, memory_gb = 7, compute_target_name='aks-mlflow-d11',autoscale_enabled=True, 
                                                autoscale_target_utilization=30,
                                                autoscale_min_replicas=3)



In [41]:
import mlflow.azureml
# Webservice creation using single command
from azureml.core.webservice import AksWebservice, Webservice

# set the model path 
model_path = "model"

(webservice,model) = mlflow.azureml.deploy( model_uri=model_path,
                      workspace=ws,
                      model_name='object_detection_model', 
                      service_name='aml-mlflow-test-aks-d11', 
                      deployment_config=aks_config, 
                      tags=None, mlflow_home=None, synchronous=True)


webservice.wait_for_deployment()


2021-01-19 18:06:49,119 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /azureml/LocalUpload/210119T180646-13afcb01/model/code/pyimagesearch/__init__.py?sv=2019-02-02&sr=b&sig=p7GCpapZO6IINqvDC%2F3S8tMyIZPt5VynU68ALpCPbvE%3D&st=2021-01-20T01%3A56%3A49Z&se=2021-01-21T02%3A06%3A49Z&sp=rcw&comp=blocklist&timeout=30
2021-01-19 18:06:49,127 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /azureml/LocalUpload/210119T180646-13afcb01/model/code/pyimagesearch/__pycache__/__init__.cpython-37.pyc?sv=2019-02-02&sr=b&sig=oAwgSM%2BZIzUoiBg3NLTzb4WoWVCVTgqRVrRW7FNuRUo%3D

Registering model object_detection_model


2021/01/19 18:07:13 INFO mlflow.azureml: Registered an Azure Model with name: `object_detection_model` and version: `25`
2021/01/19 18:07:20 INFO mlflow.azureml: Deploying an Azure Webservice with name: `aml-mlflow-test-aks-d11`


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
AKS service creation operation finished, operation "Succeeded"
AKS service creation operation finished, operation "Succeeded"


In [78]:
#Score model with SDK object
response = webservice.run(body)
print(response)

[[0.12486386299133301, 0.1821191906929016, 0.8843520283699036, 0.834100604057312, 'airplane'], [0.1420917510986328, 0.18152016401290894, 0.8583660125732422, 0.8296916484832764, 'airplane']]


In [80]:

#another way to call http request directly 
import requests
uri = webservice.scoring_uri
api_key = '' # Replace this with the API key for the web service

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

# uri ='http://9c729eb8-c2d1-4f24-9114-fbd35d040d7e.westus2.azurecontainer.io/score'
response = requests.post(url=uri, data=body,headers=headers)
print(response.text)


[[0.12486386299133301, 0.1821191906929016, 0.8843520283699036, 0.834100604057312, "airplane"], [0.1420917510986328, 0.18152016401290894, 0.8583660125732422, 0.8296916484832764, "airplane"]]


In [25]:
#Visualize result
import imutils
import pickle
import cv2
import os
import ast
pred_results = ast.literal_eval(response.text)
for pred_result, imagePath in zip( pred_results,test_image_paths):
	# load the input image (in Keras format) from disk and preprocess
	# it, scaling the pixel intensities to the range [0, 1]

	startX, startY, endX, endY,label = pred_result[0],pred_result[1],pred_result[2],pred_result[3],pred_result[4]

	# determine the class label with the largest predicted
	# probability

	# load the input image (in OpenCV format), resize it such that it
	# fits on our screen, and grab its dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# scale the predicted bounding box coordinates based on the image
	# dimensions
	startX = int(startX * w)
	startY = int(startY * h)
	endX = int(endX * w)
	endY = int(endY * h)

	# draw the predicted bounding box and class label on the image
	y = startY - 10 if startY - 10 > 10 else startY + 10
	cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (0, 255, 0), 2)
	cv2.rectangle(image, (startX, startY), (endX, endY),
		(0, 255, 0), 2)

	# show the output image
	cv2.imshow("Output", image)
	cv2.waitKey(0)